In [21]:
import pandas as pd
import psycopg2 as pg
import panel as pn
from sqlalchemy import Enum
from sqlalchemy import create_engine, text
import uuid
from IPython.display import display, Markdown

In [22]:
conn = pg.connect(
    host="localhost",
    port=5432,
    dbname="fbd",
    user="postgres",
    password="03vvm0701",
    options="-c client_encoding=UTF8"
    )
print("Conexão bem-sucedida!")
cursor = conn.cursor()

Conexão bem-sucedida!


In [26]:
# Título Principal
titulo = pn.pane.Markdown("# 💻 Sistema de Gerenciamento da Plataforma Bet UFC")

# Criar placeholders para cada CRUD
crud_usuarios = pn.Column("## 👤 Gerenciamento de Usuários", visible=False)
crud_administradores = pn.Column("## 🔑 Gerenciamento de Administradores", visible=False)
crud_eventos = pn.Column("## ⚽ Gerenciamento de Eventos Esportivos", visible=False)
crud_apostas = pn.Column("## 🎲 Gerenciamento de Apostas", visible=False)
crud_transacoes = pn.Column("## 💰 Gerenciamento de Transações", visible=False)
crud_relatorios = pn.Column("## 📊 Gerenciamento de Relatórios", visible=False)

# Função para esconder todos os CRUDs
def esconder_todos():
    crud_usuarios.visible = False
    crud_administradores.visible = False
    crud_eventos.visible = False
    crud_apostas.visible = False
    crud_transacoes.visible = False
    crud_relatorios.visible = False

# Funções para mostrar cada CRUD
def mostrar_usuarios(event=None):
    esconder_todos()
    crud_usuarios.visible = True

def mostrar_administradores(event=None):
    esconder_todos()
    crud_administradores.visible = True

def mostrar_eventos(event=None):
    esconder_todos()
    crud_eventos.visible = True

def mostrar_apostas(event=None):
    esconder_todos()
    crud_apostas.visible = True

def mostrar_transacoes(event=None):
    esconder_todos()
    crud_transacoes.visible = True

def mostrar_relatorios(event=None):
    esconder_todos()
    crud_relatorios.visible = True

# Criar botões do Menu
menu = pn.Row(
    pn.widgets.Button(name="👤 Usuários", button_type="primary", width=150, on_click=mostrar_usuarios),
    pn.widgets.Button(name="🔑 Administradores", button_type="primary", width=150, on_click=mostrar_administradores),
    pn.widgets.Button(name="⚽ Eventos", button_type="primary", width=150, on_click=mostrar_eventos),
    pn.widgets.Button(name="🎲 Apostas", button_type="primary", width=150, on_click=mostrar_apostas),
    pn.widgets.Button(name="💰 Transações", button_type="primary", width=150, on_click=mostrar_transacoes),
    pn.widgets.Button(name="📊 Relatórios", button_type="primary", width=150, on_click=mostrar_relatorios),
)

# Exemplo de conteúdo básico para cada CRUD
crud_usuarios.extend([
    input_id,
    input_nome,
    input_email,
    input_senha,
    button_create,
    feedback,
    button_fetch,
    output,
    admin_feedback,
    demote_feedback,

    "### Editar Usuário",
    pn.Row(edit_id, edit_nome, edit_email, edit_senha),
    button_edit,
    edit_feedback,

    "### Deletar Usuário",
    delete_id,
    button_delete,
    delete_feedback
])
crud_administradores.append(pn.pane.Markdown("### Aqui ficará o CRUD completo de Administradores"))
crud_eventos.append(pn.pane.Markdown("### Aqui ficará o CRUD completo de Eventos Esportivos"))
crud_apostas.append(pn.pane.Markdown("### Aqui ficará o CRUD completo de Apostas"))
crud_transacoes.append(pn.pane.Markdown("### Aqui ficará o CRUD completo de Transações"))
crud_relatorios.extend([
    "## Gerenciamento de Relatórios",
    pn.Row(input_id, input_tipo, input_arquivo, input_administrador_id, input_apostador_id),
    button_create,
    feedback,
    button_fetch,
    output,
    "### Editar Relatório",
    pn.Row(edit_id, edit_tipo, edit_arquivo),
    pn.widgets.Button(name="Atualizar Relatório", button_type="warning", on_click=update_report),
    edit_feedback,
    "### Deletar Relatório",
    delete_id,
    button_delete,
    delete_feedback
])

# Layout final
app = pn.Column(
    titulo,
    menu,
    crud_usuarios,
    crud_administradores,
    crud_eventos,
    crud_apostas,
    crud_transacoes,
    crud_relatorios
)

app.show()

Launching server at http://localhost:63062


In [24]:
pn.extension('ipywidgets')

# Criando um espaço vazio inicialmente
output = pn.Column()
feedback = pn.pane.Markdown("")
edit_feedback = pn.pane.Markdown("")  # Feedback específico para edição
delete_feedback = pn.pane.Markdown("")  # Feedback para exclusão
admin_feedback = pn.pane.Markdown("")  # Feedback para tornar admin
demote_feedback = pn.pane.Markdown("")  # Feedback para rebaixar usuário

# Criando inputs (ID será preenchido automaticamente e desativado)
input_id = pn.widgets.TextInput(name="ID", placeholder="Gerado automaticamente", disabled=True)
input_nome = pn.widgets.TextInput(name="Nome", placeholder="Digite o Nome")
input_email = pn.widgets.TextInput(name="Email", placeholder="Digite o Email")
input_senha = pn.widgets.PasswordInput(name="Senha", placeholder="Digite a Senha")  # Oculta a senha

# Inputs para edição
edit_id = pn.widgets.TextInput(name="ID para Editar", placeholder="Digite o ID")
edit_nome = pn.widgets.TextInput(name="Novo Nome", placeholder="Novo Nome (opcional)")
edit_email = pn.widgets.TextInput(name="Novo Email", placeholder="Novo Email (opcional)")
edit_senha = pn.widgets.PasswordInput(name="Nova Senha", placeholder="Nova Senha (opcional)")

# Input para deletar usuário
delete_id = pn.widgets.TextInput(name="ID para Deletar", placeholder="Digite o ID do usuário")

# Função para buscar os usuários
def get_all_users(event=None):
    try:
        conn.rollback()
        cursor.execute('SELECT id, nome, email FROM "Usuario";')
        rows = cursor.fetchall()

        if rows:
            df = pd.DataFrame(rows, columns=["ID", "Nome", "Email"])
            layout = pn.Column()
            
            for _, row in df.iterrows():
                btn_admin = pn.widgets.Button(name="Tornar Administrador", button_type="primary", width=180)
                btn_admin.on_click(lambda event, user_id=row["ID"]: tornar_administrador(user_id))
                
                btn_demote = pn.widgets.Button(name="Rebaixar", button_type="danger", width=100)
                btn_demote.on_click(lambda event, user_id=row["ID"]: rebaixar_administrador(user_id))
                
                layout.append(pn.Row(row["ID"], row["Nome"], row["Email"], btn_admin, btn_demote))
            
            output.objects = [layout]
        else:
            output.objects = [pn.pane.Markdown("### Nenhum usuário encontrado.")]

    except Exception as e:
        output.objects = [pn.pane.Markdown(f"**Erro ao executar a consulta:** {e}")]

def create_user(event=None):
    try:
        # Gerar UUID automaticamente
        user_id = str(uuid.uuid4())
        input_id.value = user_id  # Preencher o campo ID automaticamente

        nome = input_nome.value.strip()
        email = input_email.value.strip()
        senha = input_senha.value.strip()

        # Verificando se todos os campos foram preenchidos
        if not (nome and email and senha):
            feedback.object = "❌ **Todos os campos são obrigatórios!**"
            return

        conn.rollback()  # Evita erro de transação abortada
        cursor.execute(
            'INSERT INTO "Usuario" (id, nome, email, senha) VALUES (%s, %s, %s, %s);',
            (user_id, nome, email, senha)
        )
        conn.commit()
        feedback.object = "✅ **Usuário criado com sucesso!**"

        # Limpar os campos após inserção (menos o ID)
        input_nome.value = ""
        input_email.value = ""
        input_senha.value = ""

        # Atualizar a tabela
        get_all_users()

    except Exception as e:
        feedback.object = f"❌ **Erro ao criar usuário:** {e}"

# Função para editar um usuário
def edit_user(event=None):
    try:
        user_id = edit_id.value.strip()
        new_nome = edit_nome.value.strip()
        new_email = edit_email.value.strip()
        new_senha = edit_senha.value.strip()

        # ID não pode estar vazio
        if not user_id:
            edit_feedback.object = "❌ **O ID é obrigatório para edição!**"
            return

        # Criar a query dinamicamente
        updates = []
        values = []

        if new_nome:
            updates.append("nome = %s")
            values.append(new_nome)
        if new_email:
            updates.append("email = %s")
            values.append(new_email)
        if new_senha:
            updates.append("senha = %s")
            values.append(new_senha)

        if not updates:
            edit_feedback.object = "⚠️ **Nenhum campo para atualizar foi preenchido!**"
            return
        # Construir query final
        values.append(user_id)
        query = f'UPDATE "Usuario" SET {", ".join(updates)} WHERE id = %s;'

        conn.rollback()
        cursor.execute(query, values)
        conn.commit()

        # Verificar se alguma linha foi realmente alterada
        if cursor.rowcount == 0:
            edit_feedback.object = "❌ **Nenhum usuário encontrado com esse ID!**"
        else:
            edit_feedback.object = "✅ **Usuário atualizado com sucesso!**"

        # Atualizar a tabela
        get_all_users()

        # Limpar os inputs de edição (menos o ID)
        edit_nome.value = ""
        edit_email.value = ""
        edit_senha.value = ""

    except Exception as e:
        edit_feedback.object = f"❌ **Erro ao editar usuário:** {e}"

# Função para deletar um usuário
def delete_user(event=None):
    try:
        user_id = delete_id.value.strip()
        conn.rollback()
        cursor.execute('SELECT COUNT(*) FROM "Administrador" WHERE "usuarioId" = %s;', (user_id,))
        exists = cursor.fetchone()[0]
        
        if exists:
            delete_feedback.object = f'❌ **O usuário {user_id} é um administrador e não pode ser excluído!**'
            return


        # Verifica se o ID foi preenchido
        if not user_id:
            delete_feedback.object = "❌ **O ID é obrigatório para excluir um usuário!**"
            return

        conn.rollback()
        cursor.execute('DELETE FROM "Usuario" WHERE id = %s;', (user_id,))
        conn.commit()

        # Verificar se alguma linha foi realmente deletada
        if cursor.rowcount == 0:
            delete_feedback.object = "❌ **Nenhum usuário encontrado com esse ID!**"
        else:
            delete_feedback.object = "✅ **Usuário excluído com sucesso!**"

        # Atualizar a tabela
        get_all_users()

        # Limpar o campo de ID para exclusão
        delete_id.value = ""

    except Exception as e:
        delete_feedback.object = f"❌ **Erro ao excluir usuário:** {e}"

# Função para tornar um usuário administrador
def tornar_administrador(user_id):
    try:
        conn.rollback()
        cursor.execute('SELECT COUNT(*) FROM "Administrador" WHERE "usuarioId" = %s;', (user_id,))
        exists = cursor.fetchone()[0]
        
        if exists:
            admin_feedback.object = f'⚠️ **O usuário {user_id} já é um administrador!**'
            return
        
        admin_id = str(uuid.uuid4())
        cursor.execute('INSERT INTO "Administrador" ("id", "usuarioId") VALUES (%s, %s);', (admin_id, user_id))
        conn.commit()
        admin_feedback.object = f'✅ **Usuário {user_id} agora é um administrador!**'
    except Exception as e:
        admin_feedback.object = f'❌ **Erro ao tornar usuário administrador:** {e}'

# Função para rebaixar um administrador
def rebaixar_administrador(user_id):
    try:
        conn.rollback()
        cursor.execute('SELECT COUNT(*) FROM "Administrador" WHERE "usuarioId" = %s;', (user_id,))
        exists = cursor.fetchone()[0]

        if not exists:
            demote_feedback.object = f'⚠️ **O usuário {user_id} não é um administrador!**'
            return

        cursor.execute('DELETE FROM "Administrador" WHERE "usuarioId" = %s;', (user_id,))
        conn.commit()
        demote_feedback.object = f'✅ **Usuário {user_id} foi rebaixado e não é mais um administrador!**'
    except Exception as e:
        demote_feedback.object = f'❌ **Erro ao rebaixar usuário:** {e}'

# Criando botões
button_fetch = pn.widgets.Button(name="Buscar Usuários", button_type="primary")
button_fetch.on_click(get_all_users)

button_create = pn.widgets.Button(name="Criar Usuário", button_type="success")
button_create.on_click(create_user)

button_edit = pn.widgets.Button(name="Editar Usuário", button_type="warning")
button_edit.on_click(edit_user)

button_delete = pn.widgets.Button(name="Deletar Usuário", button_type="danger")
button_delete.on_click(delete_user)

# Criando layout
app = pn.Column(
    "## Gerenciamento de Usuários",
    pn.Row(input_id, input_nome, input_email, input_senha),
    button_create,
    feedback,
    button_fetch,
    output,
    admin_feedback,
    demote_feedback,

    "### Editar Usuário",
    pn.Row(edit_id, edit_nome, edit_email, edit_senha),
    button_edit,
    edit_feedback,

    "### Deletar Usuário",
    delete_id,
    button_delete,
    delete_feedback
)

# Exibir no Jupyter Notebook
app.show()


C:\Users\victo\AppData\Local\Temp\ipykernel_17816\1448946121.py:1: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension('ipywidgets')


Launching server at http://localhost:63028


In [25]:
pn.extension('ipywidgets')

# Criando um espaço vazio inicialmente
output = pn.Column()
feedback = pn.pane.Markdown("")
edit_feedback = pn.pane.Markdown("")
delete_feedback = pn.pane.Markdown("")

# Criando inputs
input_id = pn.widgets.TextInput(name="ID", placeholder="Gerado automaticamente", disabled=True)
input_tipo = pn.widgets.Select(name="Tipo", options=['CREDITO', 'DEBITO'])
input_arquivo = pn.widgets.TextInput(name="Arquivo", placeholder="Nome do arquivo")
input_administrador_id = pn.widgets.TextInput(name="ID do Administrador", placeholder="Digite o ID do Administrador")
input_apostador_id = pn.widgets.TextInput(name="ID do Apostador", placeholder="Digite o ID do Apostador")

edit_id = pn.widgets.TextInput(name="ID para Editar", placeholder="Digite o ID")
edit_tipo = pn.widgets.Select(name="Novo Tipo", options=['CREDITO', 'DEBITO'])
edit_arquivo = pn.widgets.TextInput(name="Novo Arquivo", placeholder="Novo Nome do Arquivo")

delete_id = pn.widgets.TextInput(name="ID para Deletar", placeholder="Digite o ID do Relatório")

# Função para buscar todos os relatórios
def get_all_reports(event=None):
    try:
        conn.rollback()
        cursor.execute('SELECT id, tipo, "dataGeracao", arquivo, "administradorId", "apostadorId" FROM "Relatorio";')
        rows = cursor.fetchall()

        if rows:
            df = pd.DataFrame(rows, columns=["ID", "Tipo", "Data de Geração", "Arquivo", "Administrador ID", "Apostador ID"])
            output.objects = [pn.widgets.DataFrame(df, height=300, width=800, show_index=False)]
        else:
            output.objects = [pn.pane.Markdown("### Nenhum relatório encontrado.")]
    except Exception as e:
        output.objects = [pn.pane.Markdown(f"**Erro ao buscar relatórios:** {e}")]

# Função para criar um novo relatório
def create_report(event=None):
    try:
        report_id = str(uuid.uuid4())
        input_id.value = report_id

        tipo = input_tipo.value
        arquivo = input_arquivo.value.strip()
        administrador_id = input_administrador_id.value.strip()
        apostador_id = input_apostador_id.value.strip()

        if not (tipo and administrador_id and apostador_id):
            feedback.object = "❌ **Campos obrigatórios não preenchidos!**"
            return

        conn.rollback()
        cursor.execute(
            'INSERT INTO "Relatorio" (id, tipo, "dataGeracao", arquivo, "administradorId", "apostadorId") VALUES (%s, %s, CURRENT_TIMESTAMP, %s, %s, %s);',
            (report_id, tipo, arquivo, administrador_id, apostador_id)
        )
        conn.commit()

        feedback.object = "✅ **Relatório criado com sucesso!**"
        input_tipo.value = ""
        input_arquivo.value = ""
        input_administrador_id.value = ""
        input_apostador_id.value = ""
        get_all_reports()

    except Exception as e:
        feedback.object = f"❌ **Erro ao criar relatório:** {e}"

def update_report(event=None):
    try:
        report_id = edit_id.value.strip()
        novo_tipo = edit_tipo.value
        novo_arquivo = edit_arquivo.value.strip()

        if not report_id:
            edit_feedback.object = "❌ **ID é obrigatório para atualização!**"
            return

        # Montar a query dinamicamente
        updates = []
        values = []

        if novo_tipo:
            updates.append('"tipo" = %s')
            values.append(novo_tipo)
        if novo_arquivo:
            updates.append('"arquivo" = %s')
            values.append(novo_arquivo)

        if not updates:
            edit_feedback.object = "⚠️ **Nenhum campo para atualizar foi preenchido!**"
            return

        values.append(report_id)

        query = f'UPDATE "Relatorio" SET {", ".join(updates)} WHERE "id" = %s;'

        conn.rollback()
        cursor.execute(query, values)
        conn.commit()

        if cursor.rowcount == 0:
            edit_feedback.object = "❌ **Nenhum relatório encontrado com esse ID!**"
        else:
            edit_feedback.object = "✅ **Relatório atualizado com sucesso!**"
            get_all_reports()

        # Limpar campos de edição
        edit_tipo.value = None
        edit_arquivo.value = ""

    except Exception as e:
        edit_feedback.object = f"❌ **Erro ao atualizar relatório:** {e}"

# Função para deletar um relatório
def delete_report(event=None):
    try:
        report_id = delete_id.value.strip()

        if not report_id:
            delete_feedback.object = "❌ **O ID é obrigatório para excluir um relatório!**"
            return

        conn.rollback()
        cursor.execute('DELETE FROM "Relatorio" WHERE id = %s;', (report_id,))
        conn.commit()

        if cursor.rowcount == 0:
            delete_feedback.object = "❌ **Nenhum relatório encontrado com esse ID!**"
        else:
            delete_feedback.object = "✅ **Relatório excluído com sucesso!**"
            get_all_reports()

    except Exception as e:
        delete_feedback.object = f"❌ **Erro ao excluir relatório:** {e}"

# Criando botões
button_fetch = pn.widgets.Button(name="Buscar Relatórios", button_type="primary")
button_fetch.on_click(get_all_reports)

button_create = pn.widgets.Button(name="Criar Relatório", button_type="success")
button_create.on_click(create_report)

button_delete = pn.widgets.Button(name="Deletar Relatório", button_type="danger")
button_delete.on_click(delete_report)

# Layout da aplicação
app = pn.Column(
    "## Gerenciamento de Relatórios",
    pn.Row(input_id, input_tipo, input_arquivo, input_administrador_id, input_apostador_id),
    button_create,
    feedback,
    button_fetch,
    output,
    "### Editar Relatório",
    pn.Row(edit_id, edit_tipo, edit_arquivo),
    pn.widgets.Button(name="Atualizar Relatório", button_type="warning", on_click=update_report),
    edit_feedback,
    "### Deletar Relatório",
    delete_id,
    button_delete,
    delete_feedback
)

# Exibir no Jupyter
app.show()


C:\Users\victo\AppData\Local\Temp\ipykernel_17816\3099560047.py:1: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension('ipywidgets')


Launching server at http://localhost:63031
